In [16]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [17]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class3.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [18]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Total valid lines of data: 100
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html
https://linuxhint.com/pandas-compare-two-dataframes-row-by-row/
pandas.drop 
https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bool.html 

--------------------
How to remove D\n 
How to get the incorrect answer to a new dataframe
How to 

In [24]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
#print(answer_key_table)
for grade in grades: 
        value = grade.split(",")
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        #print(answer_table)
        comparison = answer_key_table.compare(answer_table,align_axis=0,keep_shape=True)
        comparison_table = pd.DataFrame(comparison)
        gradeCheck = comparison_table.value_counts(dropna=False)
        gradeCheck.to_frame()
        score  = gradeCheck[0]
        correctAnswer = score*2
        print(correctAnswer)
        

80
64
84
76
60
76
80
80
60
72
60
84
80
72
72
76
76
84
68
76
84
72
64
84
84
76
76
80
80
84
84
68
80
72
80
68
88
84
84
72
80
80
76
84
64
64
84
72
72
92
80
88
68
84
76
80
68
76
80
84
84
68
80
60
76
68
72
68
80
60
80
80
80
84
80
64
72
72
84
64
84
76
68
76
76
68
92
64
80
72
68
80
80
72
72
72
84
76
76
84


Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'